In [8]:
import requests
import json
import csv
import emoji
import re
import os
from googleapiclient.discovery import build
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

In [6]:
load_dotenv()

True

In [9]:
# Inisialisasi API Google Cloud

API_KEY = os.getenv('GOOGLE_API_KEY')
youtube = build('youtube', 'v3', developerKey=API_KEY)

In [10]:
video_url = input('Masukkan URL video YouTube: ')
video_id = video_url[-11:]
print("ID video: " + video_id)

ID video: 1qDL8bPuj_I


In [11]:
video_response = youtube.videos().list(
    part='snippet',
    id=video_id
).execute()

In [12]:
video_snippet = video_response['items'][0]['snippet']
uploader_channel_id = video_snippet['channelId']
print("ID channel: " + uploader_channel_id)

ID channel: UCC_OYI6VZtuEZuq49Ht-cQQ


In [14]:
print("Mengambil komentar...")
comments = []
nextPageToken = None
while len(comments) < 10000:
    request = youtube.commentThreads().list(
        part='snippet',
        videoId=video_id,
        maxResults=100,  # Dapat mengambil hingga 100 komentar per permintaan
        pageToken=nextPageToken
    )
    response = request.execute()
    for item in response['items']:
        comment = item['snippet']['topLevelComment']['snippet']
        # Periksa apakah komentar tersebut bukan dari pengunggah video
        if comment['authorChannelId']['value'] != uploader_channel_id:
            comment_text = comment['textDisplay']
            comment_author = comment['authorDisplayName']
            comment_published_at = comment['publishedAt']  # Tambahkan waktu publikasi komentar
            comments.append((comment_author, comment_text, comment_published_at))
    nextPageToken = response.get('nextPageToken')

    if not nextPageToken:
        break

Mengambil komentar...


In [15]:
print("Menyaring komentar...")
hyperlink_pattern = re.compile(
    r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
)

threshold_ratio = 0.65
relevant_comments = []

for author, comment_text, published_at in comments:
    comment_text = comment_text.lower().strip()
    emojis = emoji.emoji_count(comment_text)
    text_characters = len(re.sub(r'\s', '', comment_text))

    if (any(char.isalnum() for char in comment_text)) and not hyperlink_pattern.search(comment_text):
        if emojis == 0 or (text_characters / (text_characters + emojis)) > threshold_ratio:
            relevant_comments.append((author, comment_text, published_at))

Menyaring komentar...


In [16]:
for comment in relevant_comments[:8000]:
    print(f"Author: {comment[0]}, Comment: {comment[1]}, Published at: {comment[2]}")

50 komentar relevan pertama:
Author: @ulfianaini, Comment: zonasi gak berlaku di sekolah negeri favorit, rela bayar mahal agar anaknya dapet kursi yg disertai fasilitas berac atau kipas angin lebih banyak. bahkan ada yang memalsukan kartu keluarga wkwkwkwk, Published at: 2024-12-16T04:12:04Z
Author: @somecomfort, Comment: test mtk, 7x8 berapa? jangan kelamaan mikirnya tapi wk, Published at: 2024-12-15T08:06:03Z
Author: @nanas2968, Comment: oke aplikasikan coba kemampuan literasi yg diibicarakan di video: a mendapat soal pertanyaan dalam bahasa arab, a tidak memiliki kemampuan membaca bahasa arab/kemampuan berbahasa arab, bagaimana cara a memahami konteks pertanyaan? ya ga tau!!! <br>aplikasikan lagi kemampuan mengaplikasikan perhitungan yg dibahas di video: suatu perusahaan ingin membangun sebuah gedung bertingkat dengan kemiringan tertentu agar aesthetic dengan bekerjasama pada penyedia jasa tsb penyedia jasa mulai membangun tanpa perhitungan perbandingan  bahan&quot; yg dipakai dan t

In [18]:
# Simpan ke file CSV

with open('./data/youtube_comments.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Tulis header
    writer.writerow(['Author', 'Comment', 'Published'])
    # Tulis data komentar
    for comment in comments:
        writer.writerow([comment[0], comment[1], comment[2]])